# Load Datafile

This notebook is to do an end-to-end loading of a user specified data file and showcase all the 
various resources that gets populated. In this notebook, we are specifically calling out the core
set of stored procedures that will be involved in the ingestion and not the DAG itself.

#### Pre-requisite
It is assumed that the setup steps are completed successfully. These are:
 - creating the database, schemas, stages
 - defining the external stage
 - defining the functions and stored procedures
 - defining the external tables and views

If you had not done this, use the streamlit (./bin/run_app.sh) to create these as defined in the subpage 'Setup'

In [1]:
# Initialization block
from IPython.display import display, HTML, Image , Markdown
from snowflake.snowpark.session import Session
import snowflake.snowpark.types as T
import snowflake.snowpark.functions as F
import os ,configparser ,json ,logging

# Import the commonly defined utility scripts using
# dynamic path include
import sys
sys.path.append('../python/lutils')
import sflk_base as L

display(Markdown("### Initialization"))
logging.basicConfig(stream=sys.stdout, level=logging.ERROR)

# Source various helper functions
%run ./scripts/notebook_helpers.py

# Define the project home directory, this is used for locating the config.ini file
PROJECT_HOME_DIR = '../../'
config = L.get_config(PROJECT_HOME_DIR)
sp_session = L.connect_to_snowflake(PROJECT_HOME_DIR)

if(sp_session == None):
    raise Exception(f'Unable to connect to snowflake. Validate connection information ')

sp_session.use_role(f'''{config['APP_DB']['role']}''')
sp_session.use_schema(f'''{config['APP_DB']['database']}.{config['APP_DB']['schema']}''')
sp_session.use_warehouse(f'''{config['SNOW_CONN']['warehouse']}''')

df = sp_session.sql('select current_user() ,current_role() ,current_database() ,current_schema();').to_pandas()
display(df)

### Initialization

,CURRENT_USER(),CURRENT_ROLE(),CURRENT_DATABASE(),CURRENT_SCHEMA()
0,SOLNDEMOUSR,PUBLIC,INDSOL_CMSGOV_PRICING,PUBLIC


In [2]:
import pandas as pd
import os ,re

pd.set_option('display.max_colwidth', None)

def get_basename_of_datafile(p_datafile:str) -> str:
    base = os.path.basename(p_datafile)
    fl_base = os.path.splitext(base)
    return fl_base[0]

def get_cleansed_file_basename(p_datafile):
    fl_basename = get_basename_of_datafile(p_datafile)
    # Replace all non alphanumeric characters with _
    fl_name = re.sub('[^0-9a-zA-Z]+', '_', fl_basename)
    return fl_name

INPUT_DATA_STAGE = config['APP_DB']['ext_stage']
# INPUT_DATA_STAGE = 'data_stg'
DATA_STAGE_FOLDER = config['APP_DB']['folder_data']

# DATA_FILE = '2022_10_01_priority_health_HMO_in-network-rates.zip'
DATA_FILE = '2023-02-01_KFHP_SC-COMMERCIAL_in-network-rates.zip'
# DATA_FILE = 'reduced_sample_data.json'
# DATA_FILE = '2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz'

SEGMENTS_PER_TASK = 10 #15000

DATA_FILE_BASENAME = get_basename_of_datafile(DATA_FILE)
DATA_FILE_BASENAME_CLEANSED = get_cleansed_file_basename(DATA_FILE)

TARGET_DATA_STAGE = config['APP_DB']['ext_stage']
TARGET_FOLDER = config['APP_DB']['folder_parsed']

print(f'Input DataFile: @{INPUT_DATA_STAGE}/{DATA_STAGE_FOLDER}/{DATA_FILE}')
print(f'Target: @{TARGET_DATA_STAGE}/{TARGET_FOLDER}')

Input DataFile: @ext_data_stg/data/2023-02-01_KFHP_SC-COMMERCIAL_in-network-rates.zip
Target: @ext_data_stg/raw_parsed_v3


In [20]:
# display(Markdown("### Cleanup block"))
# # We will cleanup specific resources and artifacts from possible previous runs.

# stmts = [
#     f''' truncate table segment_task_execution_status; '''
#     ,f''' truncate table task_to_segmentids; '''
#     ,f''' truncate table in_network_rates_file_header; '''
#     ,f''' truncate table in_network_rates_segment_header; '''
#     ,f''' alter stage {INPUT_DATA_STAGE} refresh; '''
# ]    
    
# print(' truncating tables ...')
# for stmt in stmts:
#     sp_session.sql(stmt).collect()

# print(f''' cleaning up files in external stage under path {DATA_STAGE_FOLDER}/{DATA_FILE_BASENAME}/ ...''')

# stmt = f''' select relative_path from directory(@{DATA_STAGE}) where relative_path like '%{DATA_STAGE_FOLDER}/{DATA_FILE_BASENAME}/%'; '''
# files = sp_session.sql(stmt).collect()
# for r in files:
#     stmt = f''' remove @{DATA_STAGE}/{r['RELATIVE_PATH']}; '''
#     sp_session.sql(stmt).collect()

---
## Data loading
We will be loading the segments and also file header and show case how this functions without the dag.

In [21]:
df = sp_session.call('parse_negotiation_arrangement_segments' 
    ,f'{INPUT_DATA_STAGE}/{DATA_STAGE_FOLDER}' 
    ,DATA_FILE ,f'@{TARGET_DATA_STAGE}/{TARGET_FOLDER}'
    ,0 ,SEGMENTS_PER_TASK)

sp_session.sql(f''' alter stage {TARGET_DATA_STAGE} refresh; ''').collect()
print(' Status of execution')
print(df)

ERROR:snowflake.snowpark._internal.server_connection:Failed to execute query [queryID: 01a98adb-0402-5fc7-0057-3c030207a17a] CALL parse_negotiation_arrangement_segments('ext_data_stg/data', '2022-12-01_cigna-health-life-insurance-company_national-ppo_in-network-rates.json.gz', '@ext_data_stg/raw_parsed', 0 :: INT, 10 :: INT)
100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/home/udf/374668997641/negotiation_arrangements.py", line 365, in main
    refresh_stages_and_tables(p_session ,p_target_stage)
  File "/home/udf/374668997641/negotiation_arrangements.py", line 317, in refresh_stages_and_tables
    p_session.sql(f'alter stage {target_stg} refresh; ').collect()
  File "/usr/lib/python_udf/2167b14ca8301f58f6da842c614dcc8f357d41d0a4f4c6a181ac35dfb2ea4cb8/lib/python3.8/site-packages/snowflake/snowpark/_internal/telemetry.py", line 138, in wrap
    result = func(*args, **kwargs)
  File "/usr/lib/python_udf/2167b14ca8301f58f6da842c614dcc8f357d41d0a4f4c6a

SnowparkSQLException: (1304): 01a98adb-0402-5fc7-0057-3c030207a17a: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "/home/udf/374668997641/negotiation_arrangements.py", line 365, in main
    refresh_stages_and_tables(p_session ,p_target_stage)
  File "/home/udf/374668997641/negotiation_arrangements.py", line 317, in refresh_stages_and_tables
    p_session.sql(f'alter stage {target_stg} refresh; ').collect()
  File "/usr/lib/python_udf/2167b14ca8301f58f6da842c614dcc8f357d41d0a4f4c6a181ac35dfb2ea4cb8/lib/python3.8/site-packages/snowflake/snowpark/_internal/telemetry.py", line 138, in wrap
    result = func(*args, **kwargs)
  File "/usr/lib/python_udf/2167b14ca8301f58f6da842c614dcc8f357d41d0a4f4c6a181ac35dfb2ea4cb8/lib/python3.8/site-packages/snowflake/snowpark/dataframe.py", line 549, in collect
    return self._internal_collect_with_tag_no_telemetry(
  File "/usr/lib/python_udf/2167b14ca8301f58f6da842c614dcc8f357d41d0a4f4c6a181ac35dfb2ea4cb8/lib/pyth

In [6]:
print(' Parsing file headers')
df = sp_session.call('parse_file_header' ,f'{INPUT_DATA_STAGE}/{DATA_STAGE_FOLDER}',DATA_FILE)

print(' Status of execution')
print(df)

 Parsing file headers
 Status of execution
{
  "data_file": "2022_10_01_priority_health_HMO_in-network-rates.zip",
  "elapsed": "=> 0:14:33.569934 ",
  "status": true
}


In [5]:
print(' Parsing segment headers')
df = sp_session.call('negotiation_arrangements_header' ,f'{INPUT_DATA_STAGE}/{DATA_STAGE_FOLDER}',DATA_FILE)

print(' Status of execution')
print(df)

 Parsing segment headers
 Status of execution
{
  "cleansed_data_file_basename": "2022_10_01_priority_health_HMO_in_network_rates",
  "data_file": "2022_10_01_priority_health_HMO_in-network-rates.zip",
  "data_file_basename": "2022_10_01_priority_health_HMO_in-network-rates",
  "elapsed": "=> 0:18:17.641089 ",
  "segments_count": 6943,
  "status": true
}


In [3]:
print(' Parsing provider references')
df = sp_session.call('provider_references' ,f'{INPUT_DATA_STAGE}/{DATA_STAGE_FOLDER}',DATA_FILE)

print(' Status of execution')
print(df)

 Parsing provider references
 Status of execution
{
  "cleansed_data_file_basename": "2023_02_01_KFHP_SC_COMMERCIAL_in_network_rates",
  "data_file": "2023-02-01_KFHP_SC-COMMERCIAL_in-network-rates.zip",
  "data_file_basename": "2023-02-01_KFHP_SC-COMMERCIAL_in-network-rates",
  "elapsed": "=> 0:02:15.156716 ",
  "segments_count": 9055,
  "status": true
}


---
## Inspection

In [ ]:
display(Markdown("The table in_network_rates_file_header holds the file header elements"))

df = (sp_session.table('in_network_rates_file_header')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)

In [ ]:
print(' The HEADER has the following data ')
json.loads(df['HEADER'][0])

In [ ]:
display(Markdown("The table in_network_rates_segment_header lists the various segments that were loaded"))

df = (sp_session.table('in_network_rates_segment_header')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)

In [ ]:
print(' The NEGOTIATED_RATES_INFO has the following data ')
json.loads(df['NEGOTIATED_RATES_INFO'][0])

In [ ]:
para = f''' 
The parsed data are stored as parquet files in the external stage at: @{TARGET_DATA_STAGE}/{TARGET_FOLDER}. The directory structure follows the format:

<ext stage>/<folder_parsed>/<data file basename (ex: reduced_sample_data)/<segment_id>/<segment type (negotiated_rates)>/data_<seq_no>_<chunk_no>.parquet.gz

Description:
- ext stage : external stage name
- folder_parsed : configured value of APP_DB.folder_parsed in config.ini
- data file basename : the data file basename, without the file extension
- segment_id : a unique identifier for the segment, this is a composite key of <negotiation_arrangement>::<billing_code_type>::<billing_code>::<billing_code_type_version>
- segment type : indicates the segment children type, this could be either of negotiated_rates/bundled_codes/covered_services
- The file which will contain the record, stored in parquet file.
'''
print(para)

DATA_FILE_BASENAME = get_basename_of_datafile(DATA_FILE)
stmt = f''' select relative_path from directory(@{TARGET_DATA_STAGE}) where relative_path like '%{TARGET_FOLDER}/{DATA_FILE_BASENAME}%' limit 5; '''
df = sp_session.sql(stmt).to_pandas()
display(df)

In [ ]:
para = f''' 
The parquet file can be queried inside Snowflake via external table: ext_negotiated_arrangments_staged. Each of the folder structure is partitioned to seperate columns , which can help with pruning to specific segments.
The value contains the negotiated_rates and other children elements stored in JSON format
'''
print(para)

pd.set_option('display.max_colwidth', 25)
df = (sp_session.table('ext_negotiated_arrangments_staged')
        .filter(F.col('P_DATA_FL') == F.lit(DATA_FILE_BASENAME))
        .sample(n=5)
        .to_pandas())

display(df)


In [ ]:
print(' a sample view of one of the records')
j = json.loads(df['VALUE'][0])
print(j)

In [ ]:
display(Markdown("The table segment_task_execution_status contains the audit of execution for the above stored procedures"))

df = (sp_session.table('segment_task_execution_status')
        .filter(F.col('DATA_FILE') == F.lit(DATA_FILE))
        .sample(n=5)
        .to_pandas())

display(df)

--- 
### Closeout

    With that we are finished this section of the demo setup

In [ ]:
sp_session.close()
print('Finished!!!')